# Exploration and a Simple Baseline

In this part of the assignment, we'll introduce the [Stanford Sentiment Treebank](https://nlp.stanford.edu/sentiment/index.html) (SST) dataset, and train a Naive Bayes model as a simple baseline. The SST, introduced by [(Socher et al. 2013)](http://nlp.stanford.edu/~socherr/EMNLP2013_RNTN.pdf) consists of approximately 10,000 sentences from movie reviews associated with fine-grained sentiment labels on a five-point scale, and is a popular benchmark for text classification.

## Outline

- **Part (a):** The Stanford Sentiment Treebank
- **Part (b):** Naive Bayes
- **Part (c):** Exploring Negation

Exercises are interspersed throughout the notebook. Be sure you catch all of them! There are 4 questions for Part (a), 2 for Part (b), and 3 for Part (c).

In [2]:
# Install a few python packages using pip
from w266_common import utils
utils.require_package("wget")      # for fetching dataset
utils.require_package("bokeh")     # for plotting histograms
utils.require_package("graphviz")  # for rendering trees

### Preliminaries: GraphViz

This notebook uses [GraphViz](https://www.graphviz.org/) to render tree structures. We'll use this several times later in the course, particularly on Assignment 5, so it's worth having. On Ubuntu / Debian (including Google Cloud), you can install it by running on the command line:
```
sudo apt-get install graphviz
```

For Mac OSX, you can install using Homebrew:
```
brew install graphviz
```
or see https://www.graphviz.org/download/ for more options. Run the cell below to set up rendering and show a sample tree.

In [3]:
import nltk
from w266_common import treeviz
import sst
# Monkey-patch NLTK with better Tree display that works on Cloud or other display-less server.
print("Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.")
treeviz.monkey_patch(nltk.tree.Tree, node_style_fn=sst.sst_node_style, format='svg')

# Test rendering
print("Sample tree to test rendering:")
nltk.tree.Tree.fromstring("(4 (4 (2 I) (3 love) (3 W266)) (3 😄))")

Overriding nltk.tree.Tree pretty-printing to use custom GraphViz.
Sample tree to test rendering:


Tree('4', [Tree('4', [Tree('2', ['I']), Tree('3', ['love']), Tree('3', ['W266'])]), Tree('3', ['😄'])])

In [4]:
from __future__ import division
import os, sys, re, json, time, datetime, shutil
import itertools, collections
from importlib import reload
from IPython.display import display, HTML

# NLTK, NumPy, and Pandas.
import nltk
import numpy as np
import pandas as pd

# Helper libraries.
from w266_common import utils, vocabulary, tf_embed_viz, treeviz
from w266_common import patched_numpy_io
# Code for this assignment
import sst, models, models_test

# Bokeh for plotting.
import bokeh.plotting as bp
from bokeh.models import HoverTool
bp.output_notebook()

# Helper code for plotting histograms
def plot_length_histogram(lengths, x_range=[0,100], bins=40, normed=True):
    hist, bin_edges = np.histogram(a=lengths, bins=bins, normed=normed, range=x_range)
    bin_centers = (bin_edges[1:] + bin_edges[:-1])/2
    bin_widths =  (bin_edges[1:] - bin_edges[:-1])

    hover = HoverTool(tooltips=[("bucket", "@x"), ("count", "@top")], mode="vline")
    fig = bp.figure(plot_width=800, plot_height=400, tools=[hover])
    fig.vbar(x=bin_centers, width=bin_widths, top=hist, hover_fill_color="firebrick")
    fig.y_range.start = 0
    fig.x_range.start = 0
    fig.xaxis.axis_label = "Example length (number of tokens)"
    fig.yaxis.axis_label = "Frequency"
    bp.show(fig)

/home/jackcworkman/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Loading BokehJS ...

# Part (a): The Stanford Sentiment Treebank

_If you haven't yet, be sure to familiarize yourself with the [Prelude notebook](Prelude.ipynb), as this assignment will assume familiarity with the text pre-processing steps described there._

The [Stanford Sentiment Treebank](https://nlp.stanford.edu/sentiment/index.html) (SST) is one of the most widely used datasets as a benchmark for text classification. It consists of 11,855 sentences drawn from a corpus of movie reviews (originally from Rotten Tomatoes), each labeled with sentiment on a five-point scale.

For example:
```
sentence: [A warm , funny , engaging film .]
label:    4 (very positive)
```

For this assignment, we'll work with the binarized form of the dataset, where the lowest two classes are mapped to a single "negative" label, the highest two are mapped to a single "positive" label, and neutral examples are omitted.

**Side note:**
Unlike most classification datasets, SST is also a _treebank_, which means each sentence is associated with a tree structure that decomposes it into subphrases. So for the example above, we'd also have sentiment labels for `[warm , funny]` and `[engaging film .]` and so on. The trees are created by running the [Stanford Parser](https://nlp.stanford.edu/software/lex-parser.shtml) over the original sentences, then crowdsourcing sentiment labels on each sub-phrase. We'll talk more about treebanks later in the course, and in Assignment 5 you'll implement a simple version of the Stanford Parser.

For the purposes of this assignment, we'll mostly concern ourselves with the sentence-level ("root") labels, but the tree structure will come in handy in two places:
- As a way of analyzing the examples to find instances of negation
- (optionally) As a source of additional training data, by including phrase labels

### Obtaining the Data
The data is distributed as serialized trees in [S-expression](https://en.wikipedia.org/wiki/S-expression) form, like this:
```
(4 (4 (2 A) (4 (3 (3 warm) (2 ,)) (3 funny))) (3 (2 ,) (3 (4 (4 engaging) (2 film)) (2 .))))
```

We've provided an `SSTDataset` class (in `sst.py`) which will download the dataset and parse the S-expressions into [`nltk.tree.Tree`](http://www.nltk.org/api/nltk.html?highlight=tree#nltk.tree.Tree) objects that you can easily view in the notebook.

`SSTDataset` also implements the text-processing pipeline described in the [Prelude notebook](Prelude.ipynb), and provides methods (`as_sparse_bow` and `as_padded_array`) to convert the data to matrix form.

Run the cell below; it will download a ~6MB .zip file to the local directory the first time you run it.

In [5]:
import sst
ds = sst.SSTDataset(V=20000).process(label_scheme="binary")

Loading SST from data/sst/trainDevTestTrees_PTB.zip
Training set:     8,544 trees
Development set:  1,101 trees
Test set:         2,210 trees
Building vocabulary - 16,474 words
Processing to phrases...  Done!
Splits: train / dev / test : 98,794 / 13,142 / 26,052


A few members of the `SSTDataset()` class that you might find useful:
- **`ds.vocab`**: a `vocabulary.Vocabulary` object managing the model vocabulary
- **`ds.{train,dev,test}_trees`**: a list of [`nltk.tree.Tree`](http://www.nltk.org/api/nltk.html?highlight=tree#nltk.tree.Tree) objects representing each sentence
- **`ds.{train,dev,test}`**: a Pandas DataFrame containing the _processed_ examples, including all subphrases. `label` is the target label, `is_root` denotes whether this example is a root node (full sentence), and `root_id` is the index of the tree that the example was derived from.

In [6]:
# Look at a tree for a positive review
ds.dev_trees[3]

Tree('4', [Tree('4', [Tree('2', ['A']), Tree('4', [Tree('3', [Tree('3', ['warm']), Tree('2', [','])]), Tree('3', ['funny'])])]), Tree('3', [Tree('2', [',']), Tree('3', [Tree('4', [Tree('4', ['engaging']), Tree('2', ['film'])]), Tree('2', ['.'])])])])

In [7]:
# Look at a tree for a negative review
ds.dev_trees[361]

Tree('0', [Tree('2', ['It']), Tree('2', [Tree('2', [Tree('2', ["'s"]), Tree('0', [Tree('2', ['like']), Tree('1', [Tree('2', ['watching']), Tree('0', [Tree('1', [Tree('2', ['a']), Tree('0', ['nightmare'])]), Tree('2', [Tree('2', ['made']), Tree('2', ['flesh'])])])])])]), Tree('2', ['.'])])])

## Part (a) questions: Exploring the Data

Let's explore the data a bit, to get a sense of what we're working with. If you're not familiar with DataFrames, you may wish to review the Pandas documentation: https://pandas.pydata.org/pandas-docs/stable/dsintro.html 

Answer the following questions in the cells below:

1. Looking at only the root examples in the training set (*Hint: use `ds.train[ds.train.is_root]`*), what is the fraction of positive labels? Is the classification task balanced, or close to it? If we used most-common-class as a baseline classifier, what would the accuracy be?
2. What are the five most common tokens in the dataset? (*Hint: there are several ways to get at this - you might use `collections.Counter`, or poke around in the `ds.vocab` object.*)
3. Use the `plot_length_histogram` function (defined above) to plot a histogram of the sentence lengths. What is the 95% percentile length? (i.e. 95% of examples in the training set should be shorter than this)
4. Repeat 3., but this time including all subphrases. Notice the difference in distributions. Could this be problematic, if we include subphrases in our training data?  (Why?)

Use the cells below for your code solutions. Each part shouldn't require more than a couple lines, but you're welcome to explore more!

In [8]:
#### YOUR CODE HERE ####
# Code for Part (a).1
# Looking at only the root examples in the training set (Hint: use ds.train[ds.train.is_root]),
total = len(ds.train[ds.train.is_root])
positives = len(ds.train[ds.train.is_root][ds.train['label'] == 1])
#print(total)
#print(positives)
#print(total - positives)
# what is the fraction of positive labels?
print('Fraction of positive labels = {0:.2f}'.format(positives / total))
# Is the classification task balanced, or close to it?
print('The classifciation task is close to balanced.')
# If we used most-common-class as a baseline classifier, what would the accuracy be?
print('Assuming "most-common-class" classifier means that the most common class is always chosen, the accuracy would be {0:.2f} because the most common label is postive.'
      .format(positives / total))
#### END(YOUR CODE) ####

Fraction of positive labels = 0.52
The classifciation task is close to balanced.
Assuming "most-common-class" classifier means that the most common class is always chosen, the accuracy would be 0.52 because the most common label is postive.


/home/jackcworkman/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """


In [9]:
#### YOUR CODE HERE ####
# Code for Part (a).2
# What are the five most common tokens in the dataset?
# (Hint: there are several ways to get at this - you might use collections.Counter, or poke around in the ds.vocab object.)
print(ds.vocab.unigram_counts.most_common(5))
#### END(YOUR CODE) ####

[('.', 8024), ('the', 7303), (',', 7131), ('a', 5281), ('and', 4473)]


In [10]:
#### YOUR CODE HERE ####
# Code for Part (a).3
# Use the plot_length_histogram function (defined above) to plot a histogram of the sentence lengths.
sentence_lens = ds.train[ds.train.is_root]['tokens'].str.len()
plot_length_histogram(sentence_lens)
# What is the 95% percentile length? (i.e. 95% of examples in the training set should be shorter than this)
print(sentence_lens.quantile([.05, .25, .5, .75, .95]))
#### END(YOUR CODE) ####

0.05     6.0
0.25    12.0
0.50    19.0
0.75    25.0
0.95    36.0
Name: tokens, dtype: float64


In [11]:
#### YOUR CODE HERE ####
# Code for Part (a).4
# Repeat 3., but this time including all subphrases. Notice the difference in distributions.
sentence_lens = ds.train['tokens'].str.len()
plot_length_histogram(sentence_lens)
print(sentence_lens.quantile([.05, .25, .5, .75, .95]))
# Could this be problematic, if we include subphrases in our training data? (Why?)
print('No, it will hopefully give our model a finer understanding of the valence of the words in the dataset. '
      'This might help when we encounter sentences with both positive and negative words. In fact, when '
      'the part (b) example is run with all examples (not just root), the accuracy improves by 1%.')
#### END(YOUR CODE) ####

0.05     1.0
0.25     2.0
0.50     5.0
0.75    11.0
0.95    24.0
Name: tokens, dtype: float64
No, it will hopefully give our model a finer understanding of the valence of the words in the dataset. This might help when we encounter sentences with both positive and negative words. In fact, when the part (b) example is run with all examples (not just root), the accuracy improves by 1%.


# Part (b): Naive Bayes

In this section, we'll build and explore a Naive Bayes model as a baseline classifier for our dataset.

Naive Bayes is perhaps the simplest possible classification algorithm, but it's one that still surprisingly effective for many text classification problems. Recall from your Machine Learning course:

$$ P(y = k) = \hat{\theta}_k = \frac{1}{N}\sum_{i = 1}^N \mathbf{1}[y_i = k] $$

$$ P(x_j | y = 1) = \hat{\theta}_{k,j} = 
\frac{ 
\sum_{i = 1}^N  \sum_{j' = 1}^{n_i} \mathbf{1}[y_i = 1 \wedge x_{j'} = j]
}{
\sum_{i = 1}^N  \mathbf{1}[y_i = 1] \cdot n_i
}
$$

where $N$ is the size of the dataset, and $n_i$ is the length (number of tokens of the $i^{th}$ example. Prediction is done by computing the score:

$$ \mathrm{score}(x) = \log \left(\frac{P(y = 1) \prod_{j=1}^n P(x_j | y = 1)}{P(y = 0) \prod_{j=1}^n P(x_j | y = 0)}\right) $$

We'll just use the [implementation from scikit-learn](http://scikit-learn.org/stable/modules/naive_bayes.html). Like other scikit-learn classifiers, this expects the input as a `scipy.sparse` matrix. Run the cell below:

In [12]:
# 'csr' stands for "Compressed Sparse Row", which is one format
# for representing sparse matricies.
train_x_csr, train_y = ds.as_sparse_bow("train", root_only=True)
test_x_csr,  test_y  = ds.as_sparse_bow("test", root_only=True)
print("Training set: x = {:s} sparse, y = {:s}".format(str(train_x_csr.shape), 
                                                str(train_y.shape)))
print("Test set:     x = {:s} sparse, y = {:s}".format(str(test_x_csr.shape), 
                                                str(test_y.shape)))

Training set: x = (6920, 16474) sparse, y = (6920,)
Test set:     x = (1821, 16474) sparse, y = (1821,)


Note the `root_only=True` parameter - this will return only examples corresponding to whole sentences. If you set this to false, you can get examples for all phrases.

## Part (b) Questions

Answer the following questions in the answer and code cells below.

**Question 1.)** Implement Naive Bayes using `sklearn.naive_bayes.MultinomialNB`. Train on the training set and evaluate accuracy on the test set using `.predict(...)`. 

Your model should train almost instantly, and score 82.21% - not bad! On SST, this is actually a very strong baseline.

Recall that Naive Bayes can be interpreted as a linear model, where score is given by:

$$ \mathrm{score}(x) = \log \left(\frac{P(y = 1) \prod_{j=1}^n P(x_j | y = 1)}{P(y = 0) \prod_{j=1}^n P(x_j | y = 0)}\right) 
= \left( \log\hat{\theta}_1 - \log\hat{\theta}_0 \right) + \sum_{j=1}^n \left( \log\hat{\theta}_{1,j} - \log\hat{\theta}_{0,j} \right)$$

You can access the values $\log\hat{\theta}_{k,j}$ from the trained model using `nb.feature_log_prob_[k,j]`.

**Question 2.)** In the cell below, compute the weights $w_j = \left( \log\hat{\theta}_{1,j} - \log\hat{\theta}_{0,j} \right)$ of the linear model, and find the top 10 most negative and most positive weights. _(Hint: use `np.argsort` to get the indices of the most extreme elements.)_ Put the most negative and most positive word in the answers file.  Do the features you found make sense for this domain?

In [13]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
#### YOUR CODE HERE ####
# Code for part (b).1
nb = MultinomialNB()
nb.fit(train_x_csr, train_y)
y_pred = nb.predict(test_x_csr)
#### END(YOUR CODE) ####
acc = accuracy_score(test_y, y_pred)
print("Accuracy on test set: {:.02%}".format(acc))

Accuracy on test set: 82.21%


In [14]:
#### YOUR CODE HERE ####
# Code for part (b).2
df_feature_counts = pd.DataFrame(nb.feature_log_prob_.T)
linear_weights = df_feature_counts[1] - df_feature_counts[0] # populate this with actual values
sorted_features = np.argsort(linear_weights)
top_negative_features = sorted_features[0:10]
top_positive_features = sorted_features[-10:]
print('Yes, these features mostly make sense for this domain. There are a few questionable ones like "car" and "disguise" on the negative side and "portrait" on the positive side, however. The rest are reasonable.')
#### END(YOUR CODE) ####

print("Most negative features:")
for idx in top_negative_features:
    print("  {:s} ({:.02f})".format(ds.vocab.id_to_word[idx], 
                                    linear_weights[idx]))
print("")
print("Most positive features:")
for idx in top_positive_features:
    print("  {:s} ({:.02f})".format(ds.vocab.id_to_word[idx], 
                                    linear_weights[idx]))

Yes, these features mostly make sense for this domain. There are a few questionable ones like "car" and "disguise" on the negative side and "portrait" on the positive side, however. The rest are reasonable.
Most negative features:
  stupid (-3.17)
  suffers (-3.08)
  unfunny (-2.97)
  mess (-2.79)
  pointless (-2.79)
  flat (-2.75)
  poorly (-2.72)
  car (-2.72)
  tiresome (-2.64)
  disguise (-2.56)

Most positive features:
  heartwarming (2.41)
  portrait (2.41)
  wonderful (2.49)
  riveting (2.49)
  touching (2.56)
  refreshing (2.63)
  inventive (2.69)
  perfectly (2.81)
  solid (3.45)
  powerful (3.53)


# Part (c): Examining Negation

While Naive Bayes performs well in the aggregate, as a linear model it's still limited in its ability to model complex phenomena in the data. Each feature - in this case, each word - contributes a weight to the total, and if the sum is $\ge 0$ we predict the example is positive. But what happens when we have an example with both positive and negative words? For instance:

```
[Brando 's performance fell short of the high standards set by his earlier work .]
[A thoughtful look at a painful incident that made headlines in 1995 .]
```

Run the cell below to evaluate your model on these examples. It should predict both as positive.

In [15]:
examples = ["Brando 's performance fell short of the high standards set by his earlier work .",
            "A thoughtful look at a painful incident that made headlines in 1995 ."]
id_lists = [ds.vocab.words_to_ids(ds.canonicalize(s.split())) for s in examples]
x = utils.id_lists_to_sparse_bow(id_lists, ds.vocab.size)
print(x.shape)
nb.predict(x)

(2, 16474)


array([1, 1], dtype=int32)

## Part (c).1
<a id="answers_c1"></a>

**Question 1.)** Why does the model get the first example wrong? Why does it get the second one correct, despite the presence of the words "painful" and "incident"?

In [16]:
df_feature_counts = pd.DataFrame(nb.feature_log_prob_.T)
linear_weights = df_feature_counts[1] - df_feature_counts[0] # populate this with actual values

    
for id_list, ex in zip(id_lists, examples):
    print('Example 1: {0}\n'.format(ex))

    total = 0
    for idx in id_list:
        weight = linear_weights[idx]
        print("  {:s} ({:.02f})".format(ds.vocab.id_to_word[idx], weight))
        total += weight
    
    print('\nweight sum = {0}\n\n'.format(total))


print("The first example demonstrates the shortcomings of a linear model in this space. "
      "The first two tokens have a very small polarity, but the third \"performance\" is very high at 1.10. "
      "The remaining words are not strong enough to bring the linear model back to below 0.")
print('For the second example, the model interprets "incident" as a positive word with a weight of 1.31.'
      'That combined with "thoughtful" at 1.91 overpower the negative words. The most negative is the "1995" token at -0.24.')

Example 1: Brando 's performance fell short of the high standards set by his earlier work .

  <unk> (-0.08)
  's (0.03)
  performance (1.10)
  fell (-0.77)
  short (-0.52)
  of (0.16)
  the (0.04)
  high (-0.08)
  standards (0.61)
  set (-0.18)
  by (-0.04)
  his (0.15)
  earlier (-1.00)
  work (0.48)
  . (0.02)

weight sum = -0.08098176055106876


Example 1: A thoughtful look at a painful incident that made headlines in 1995 .

  a (0.11)
  thoughtful (1.91)
  look (0.83)
  at (0.07)
  a (0.11)
  painful (-0.20)
  incident (1.31)
  that (0.02)
  made (-0.16)
  <unk> (-0.08)
  in (0.05)
  DGDGDGDG (-0.24)
  . (0.02)

weight sum = 3.7439184079235996


The first example demonstrates the shortcomings of a linear model in this space. The first two tokens have a very small polarity, but the third "performance" is very high at 1.10. The remaining words are not strong enough to bring the linear model back to below 0.
For the second example, the model interprets "incident" as a positive word 

It's always important to look at individual examples, but let's try to do this a bit more systematically. Recall that SST gives us labels not only at the whole-sentence (root) level, but for individual phrases as well. We can use this to look for examples where polarity changes between different parts of the sentence. Here's one of the examples above:

In [192]:
ds.test_trees[210]

Tree('3', [Tree('4', [Tree('2', ['A']), Tree('4', ['thoughtful'])]), Tree('1', [Tree('1', [Tree('2', ['look']), Tree('2', [Tree('2', ['at']), Tree('2', [Tree('1', [Tree('2', ['a']), Tree('1', [Tree('1', ['painful']), Tree('2', ['incident'])])]), Tree('2', [Tree('2', ['that']), Tree('2', [Tree('3', [Tree('2', ['made']), Tree('2', ['headlines'])]), Tree('2', [Tree('2', ['in']), Tree('2', ['1995'])])])])])])]), Tree('2', ['.'])])])

The following cell will comb through the test set, looking for examples where there's some degree of ambiguity. We'll use a fairly crude heuristic for now: count up all the non-neutral phrases for a given sentence, and look at ones where there's a mix of both positive and negative labels.

In [193]:
df = ds.test

gb = df.groupby(by=['root_id'])
interesting_ids = []   # root ids, index into ds.test_trees
interesting_idxs = []  # DataFrame indices, index into ds.test
# This groups the DataFrame by sentence
for root_id, idxs in gb.groups.items():
    # Get the average score of all the phrases for this sentence
    mean = df.loc[idxs].label.mean()
    if (mean > 0.4 and mean < 0.6):
        interesting_ids.append(root_id)
        interesting_idxs.extend(idxs)
        
print("Found {:,} interesting examples".format(len(interesting_ids)))
# This will extract only the "interesting" sentences we found above
test_x_interesting, test_y_interesting = ds.as_sparse_bow("test", root_only=True, 
                                                          df_idxs=interesting_idxs)
print("Interesting ids (into ds.test_trees): ", interesting_ids)

Found 246 interesting examples
Interesting ids (into ds.test_trees):  [0, 27, 31, 32, 75, 80, 90, 96, 117, 124, 138, 140, 141, 160, 166, 186, 187, 205, 210, 212, 227, 232, 254, 269, 271, 285, 296, 307, 312, 327, 335, 373, 397, 399, 406, 407, 410, 426, 447, 511, 512, 516, 521, 534, 539, 563, 577, 588, 606, 610, 611, 637, 640, 645, 655, 662, 664, 713, 720, 721, 724, 739, 755, 758, 763, 776, 791, 793, 796, 802, 805, 810, 818, 840, 858, 887, 898, 899, 909, 910, 912, 929, 930, 961, 970, 973, 974, 975, 979, 1008, 1032, 1036, 1066, 1067, 1076, 1098, 1101, 1108, 1114, 1131, 1138, 1142, 1159, 1183, 1185, 1189, 1193, 1198, 1206, 1214, 1215, 1235, 1241, 1243, 1244, 1261, 1267, 1273, 1275, 1279, 1280, 1293, 1296, 1302, 1303, 1312, 1318, 1319, 1321, 1322, 1324, 1326, 1328, 1338, 1341, 1346, 1359, 1363, 1371, 1383, 1398, 1402, 1413, 1443, 1452, 1456, 1458, 1462, 1464, 1480, 1481, 1486, 1487, 1488, 1507, 1509, 1513, 1516, 1527, 1537, 1552, 1576, 1582, 1587, 1594, 1597, 1602, 1607, 1608, 1615, 1619, 1

## Part (c) continued

Answer the following in the cells below.

**Question 2.)** Examine a few of the "interesting" trees. What kinds of patterns do you see? What is the relation of the polarity of the sub-phrases to that of the whole sentence? Is this well-captured by a linear model?

**Question 3.)** Evaluate your model on `test_x_interesting` and compute accuracy. Does your model do better or worse on this slice of the data than on the whole test set (interesting + uninteresting examples)? What is the _relative_ change in the error rate? _(For example, if you go from 90% accuracy to 85%, that's a 50% increase in error!)_

In [194]:
#### YOUR CODE HERE ####
# Code for part (c).2
display(ds.test_trees[406])
display(ds.test_trees[138])
display(ds.test_trees[1458])
display(ds.test_trees[1789])
display(ds.test_trees[1950])
display(ds.test_trees[1982])
display(ds.test_trees[2193])
display(ds.test_trees[2008])
print('Through observation, I notice that the majority of the leaf nodes have a value of 2, '
      'but otherwise no discernible pattern between the root and leaf nodes.\n')

def compare_sentence_to_subphrase_polarity(trees):
    wholephrase_values = list()
    subphrase_avgs = list()
    for tree_id in trees:

        tree = ds.test_trees[tree_id]

        #print('tree_id: ', tree_id)
        #print('tree.label: ', tree.label())
        #print('tree:    ', tree)
        
        wholephrase_values.append(float(tree.label()))
        subphrase_values = list()

        for subtree in tree.subtrees(lambda t: t.height() > 0 and t.height() != tree.height()):

            #print('subtree: ', subtree)
            #print('subtree.label: ', subtree.label())
            
            subphrase_values.append(float(subtree.label()))
            
        subphrase_avgs.append(np.mean(subphrase_values))

    #return np.mean(wholephrase_values), np.mean(subphrase_avgs)
    return wholephrase_values, subphrase_avgs

trees, subphrases = compare_sentence_to_subphrase_polarity(interesting_ids)
pos_pos = 0
neg_neg = 0
oppo = 0
for treeval, subval in zip(trees, subphrases):
    if treeval > 2 and subval > 2:
        pos_pos += 1
    elif treeval < 2 and subval < 2:
        neg_neg += 1
    elif (treeval > 2 and subval < 2) or (treeval < 2 and subval > 2):
        oppo += 1

print('Calculating the relationship yields:\n')
print('Whole is positive, Subphrase is positive:     ', pos_pos)
print('Whole is negative, Subphrase is negative:     ', neg_neg)
print('Whole is one polarity, Subphrase is opposite: ', oppo)

#### END(YOUR CODE) ####

Tree('1', [Tree('3', [Tree('4', [Tree('4', [Tree('2', [Tree('2', ['One']), Tree('2', [Tree('2', ['Hour']), Tree('2', ['Photo'])])]), Tree('3', [Tree('2', ['is']), Tree('3', [Tree('4', [Tree('2', ['an']), Tree('4', [Tree('3', ['intriguing']), Tree('2', ['snapshot'])])]), Tree('2', [Tree('2', ['of']), Tree('1', [Tree('2', [Tree('2', [Tree('2', ['one']), Tree('2', ['man'])]), Tree('2', ['and'])]), Tree('2', [Tree('2', ['his']), Tree('1', ['delusions'])])])])])])]), Tree('2', [';'])]), Tree('1', [Tree('2', ['it']), Tree('1', [Tree('2', ["'s"]), Tree('1', [Tree('2', ['just']), Tree('1', [Tree('2', ['too']), Tree('1', [Tree('0', ['bad']), Tree('1', [Tree('2', ['it']), Tree('1', [Tree('1', [Tree('2', ['does']), Tree('2', ["n't"])]), Tree('2', [Tree('2', ['have']), Tree('3', [Tree('2', [Tree('2', ['more']), Tree('2', ['flashes'])]), Tree('2', [Tree('2', ['of']), Tree('2', ['insight'])])])])])])])])])])])]), Tree('2', ['.'])])

Tree('3', [Tree('2', ['It']), Tree('3', [Tree('3', [Tree('2', ["'s"]), Tree('2', [Tree('2', [Tree('2', ['a']), Tree('2', [Tree('2', ['minor']), Tree('3', ['comedy'])])]), Tree('2', [Tree('2', ['that']), Tree('3', [Tree('3', [Tree('3', [Tree('2', ['tries']), Tree('2', [Tree('2', ['to']), Tree('4', [Tree('3', [Tree('2', ['balance']), Tree('3', ['sweetness'])]), Tree('1', [Tree('2', ['with']), Tree('2', ['coarseness'])])])])]), Tree('2', [','])]), Tree('2', [Tree('2', ['while']), Tree('1', [Tree('2', ['it']), Tree('1', [Tree('2', ['paints']), Tree('1', [Tree('1', [Tree('2', ['a']), Tree('1', [Tree('1', ['sad']), Tree('2', ['picture'])])]), Tree('2', [Tree('2', ['of']), Tree('2', [Tree('2', ['the']), Tree('2', [Tree('2', ['singles']), Tree('2', ['scene'])])])])])])])])])])])]), Tree('2', ['.'])])])

Tree('1', [Tree('2', [Tree('1', [Tree('2', [Tree('2', [Tree('2', [Tree('2', [Tree('2', ['Watching']), Tree('1', ['Trouble'])]), Tree('2', [Tree('2', ['Every']), Tree('2', ['Day'])])]), Tree('2', [','])]), Tree('2', [Tree('2', ['at']), Tree('1', ['least'])])]), Tree('2', [Tree('2', ['if']), Tree('3', [Tree('2', ['you']), Tree('2', [Tree('2', [Tree('2', ['do']), Tree('2', ["n't"])]), Tree('1', [Tree('2', ['know']), Tree('2', [Tree('2', ['what']), Tree('2', [Tree('2', ["'s"]), Tree('2', ['coming'])])])])])])])]), Tree('2', [','])]), Tree('0', [Tree('0', [Tree('2', ['is']), Tree('0', [Tree('2', ['like']), Tree('0', [Tree('2', [Tree('3', [Tree('2', ['biting']), Tree('3', [Tree('2', ['into']), Tree('3', [Tree('2', ['what']), Tree('3', [Tree('2', ['looks']), Tree('4', [Tree('2', ['like']), Tree('3', [Tree('3', [Tree('2', ['a']), Tree('3', [Tree('3', [Tree('3', ['juicy']), Tree('3', [Tree('2', [',']), Tree('3', ['delicious'])])]), Tree('2', ['plum'])])]), Tree('2', [Tree('2', ['on']), Tree('3', [Tree('2', ['a']), Tree('2', [Tree('4', ['hot']), Tree('2', [Tree('2', ['summer']), Tree('2', ['day'])])])])])])])])])])]), Tree('2', ['and'])]), Tree('0', [Tree('2', [Tree('2', ['coming']), Tree('2', ['away'])]), Tree('1', [Tree('2', ['with']), Tree('1', [Tree('2', [Tree('2', ['your']), Tree('2', [Tree('2', ['mouth']), Tree('3', ['full'])])]), Tree('1', [Tree('2', ['of']), Tree('1', [Tree('1', ['rotten']), Tree('2', [Tree('2', ['pulp']), Tree('2', [Tree('2', ['and']), Tree('2', [Tree('2', ['living']), Tree('2', ['worms'])])])])])])])])])])])]), Tree('2', ['.'])])])

Tree('2', [Tree('2', [Tree('2', ['Well']), Tree('2', [Tree('2', ['before']), Tree('2', [Tree('2', ['it']), Tree('2', [Tree('2', ["'s"]), Tree('2', ['over'])])])])]), Tree('2', [Tree('2', [',']), Tree('2', [Tree('2', [Tree('3', ['Beijing']), Tree('2', ['Bicycle'])]), Tree('2', [Tree('2', [Tree('2', ['begins']), Tree('0', [Tree('2', ['spinning']), Tree('2', [Tree('2', ['its']), Tree('2', ['wheels'])])])]), Tree('2', ['.'])])])])])

Tree('1', [Tree('2', [Tree('1', ['Based']), Tree('2', [Tree('2', ['on']), Tree('2', [Tree('2', ['a']), Tree('2', [Tree('2', ['David']), Tree('2', [Tree('2', ['Leavitt']), Tree('2', ['story'])])])])])]), Tree('1', [Tree('2', [',']), Tree('2', [Tree('2', [Tree('2', ['the']), Tree('2', ['film'])]), Tree('2', [Tree('3', [Tree('2', ['shares']), Tree('2', [Tree('2', [Tree('2', [Tree('2', ['that']), Tree('2', [Tree('2', ['writer']), Tree('2', ["'s"])])]), Tree('1', [Tree('2', ['usual']), Tree('2', ['blend'])])]), Tree('2', [Tree('2', ['of']), Tree('2', [Tree('2', [Tree('3', [Tree('3', [Tree('3', [Tree('2', ['observant']), Tree('3', ['cleverness'])]), Tree('2', [','])]), Tree('1', [Tree('1', ['too-facile']), Tree('2', ['coincidence'])])]), Tree('2', ['and'])]), Tree('1', [Tree('1', [Tree('2', ['slightly']), Tree('2', ['noxious'])]), Tree('3', ['preciousness'])])])])])]), Tree('2', ['.'])])])])])

Tree('0', [Tree('0', [Tree('1', [Tree('1', [Tree('1', [Tree('2', ['I']), Tree('1', [Tree('2', ['wish']), Tree('1', [Tree('2', ['I']), Tree('3', [Tree('2', ['could']), Tree('2', [Tree('2', ['say']), Tree('2', [Tree('2', ['``']), Tree('3', [Tree('3', [Tree('2', ['Thank']), Tree('3', [Tree('2', ['God']), Tree('2', ['It'])])]), Tree('3', [Tree('2', [Tree('2', ["'s"]), Tree('2', ['Friday'])]), Tree('2', ["''"])])])])])])])])]), Tree('2', [','])]), Tree('2', ['but'])]), Tree('0', [Tree('2', [Tree('2', [Tree('2', ['the']), Tree('3', ['truth'])]), Tree('2', [Tree('2', ['of']), Tree('2', [Tree('2', ['the']), Tree('2', ['matter'])])])]), Tree('1', [Tree('2', ['is']), Tree('0', [Tree('2', ['I']), Tree('0', [Tree('3', [Tree('2', ['was']), Tree('3', ['glad'])]), Tree('2', [Tree('2', ['when']), Tree('2', [Tree('2', ['it']), Tree('2', [Tree('2', ['was']), Tree('2', ['over'])])])])])])])])]), Tree('2', ['.'])])

Tree('1', [Tree('2', [Tree('2', [Tree('2', [Tree('2', [Tree('2', [Tree('3', [Tree('3', [Tree('2', [Tree('2', ['The']), Tree('2', ['film'])]), Tree('3', [Tree('2', ['has']), Tree('3', [Tree('3', [Tree('3', [Tree('2', ['a']), Tree('2', [Tree('2', ['few']), Tree('3', [Tree('3', ['cute']), Tree('2', ['ideas'])])])]), Tree('2', ['and'])]), Tree('3', [Tree('2', ['several']), Tree('3', [Tree('3', ['modest']), Tree('1', ['chuckles'])])])])])]), Tree('2', ['but'])]), Tree('1', [Tree('2', ['it']), Tree('1', [Tree('1', [Tree('2', ['is']), Tree('2', ["n't"])]), Tree('3', [Tree('2', ['exactly']), Tree('3', ['kiddie-friendly'])])])])]), Tree('2', ['...'])]), Tree('1', [Tree('2', [Tree('2', ['Alas']), Tree('2', [Tree('2', [',']), Tree('2', ['Santa'])])]), Tree('0', [Tree('2', ['is']), Tree('3', [Tree('2', ['more']), Tree('1', [Tree('2', ['ho-hum']), Tree('2', [Tree('2', ['than']), Tree('2', ['ho-ho-ho'])])])])])])]), Tree('2', ['and'])]), Tree('1', [Tree('2', [Tree('2', [Tree('2', ['the']), Tree('2', ['Snowman'])]), Tree('3', [Tree('1', ['-LRB-']), Tree('2', [Tree('2', [Tree('2', ['who']), Tree('2', [Tree('2', ['never']), Tree('2', [Tree('2', ['gets']), Tree('2', [Tree('2', ['to']), Tree('3', [Tree('2', ['play']), Tree('2', [Tree('2', ['that']), Tree('2', ['flute'])])])])])])]), Tree('3', ['-RRB-'])])])]), Tree('0', [Tree('2', ['has']), Tree('1', [Tree('3', [Tree('2', ['all']), Tree('2', [Tree('2', ['the']), Tree('3', ['charm'])])]), Tree('2', [Tree('2', ['of']), Tree('1', [Tree('2', ['a']), Tree('2', ['meltdown'])])])])])])]), Tree('2', ['.'])])

Tree('2', [Tree('2', [Tree('2', ['As']), Tree('3', [Tree('2', [Tree('2', ['is']), Tree('2', [Tree('2', ['most']), Tree('1', ['commonly'])])]), Tree('3', [Tree('2', ['case']), Tree('2', [Tree('2', ['with']), Tree('3', [Tree('2', ['projects']), Tree('3', [Tree('2', ['such']), Tree('3', [Tree('3', ['noble']), Tree('2', [Tree('2', ['and']), Tree('2', [Tree('3', ['lofty']), Tree('2', ['ambitions'])])])])])])])])])]), Tree('1', [Tree('2', [',']), Tree('1', [Tree('2', [Tree('2', ['the']), Tree('2', ['film'])]), Tree('1', [Tree('1', [Tree('2', ['is']), Tree('2', [Tree('1', [Tree('2', ['less']), Tree('3', ['poetic'])]), Tree('2', [Tree('2', ['than']), Tree('0', [Tree('2', ['simply']), Tree('1', ['pretentious'])])])])]), Tree('2', ['.'])])])])])

Through observation, I notice that the majority of the leaf nodes have a value of 2, but otherwise no discernible pattern between the root and leaf nodes.

Calculating the relationship yields:

Whole is positive, Subphrase is positive:      31
Whole is negative, Subphrase is negative:      26
Whole is one polarity, Subphrase is opposite:  81


In [210]:
#### YOUR CODE HERE ####
# Code for part (c).3
# Evaluate your model on `test_x_interesting` and compute accuracy.
y_pred = nb.predict(test_x_interesting)
acc = accuracy_score(test_y_interesting, y_pred)  # replace this with a real value for accuracy
# Does your model do better or worse on this slice of the data than on the whole test set (interesting + uninteresting examples)?
# What is the _relative_ change in the error rate?
#  _(For example, if you go from 90% accuracy to 85%, that's a 50% increase in error!)_
y_pred = nb.predict(test_x_csr)
acc_full_test = accuracy_score(test_y, y_pred)
print('Accuracy of whole set: {:.02f}%'.format(100 * acc_full_test))
if acc > acc_full_test:
    print('The model does better on the interesting examples than the whole.')
elif acc == acc_full_test:
    print('The model has the same accuracy for both sets.')
else:
    print('The model does worse on the interesting examples than the whole.')
full_error = 1 - acc_full_test
int_error = 1 - acc
print('The whole set yields an error rate of {:.02f}%'.format(100 * full_error))
print('The interesting set yields an error rate of {:.02f}%'.format(100 * int_error))
print('That is a {:.02f}% increase in error!'.format(100 * (int_error / full_error) - 100))
print(int_error / full_error)
print(full_error / int_error)
#### END(YOUR CODE) ####
print("Accuracy on selected examples: {:.02f}%".format(100*acc))

Accuracy of whole set: 82.21%
The model does worse on the interesting examples than the whole.
The whole set yields an error rate of 17.79%
The interesting set yields an error rate of 26.74%
That is a 50.31% increase in error!
1.503122308354867
0.6652818566005299
Accuracy on selected examples: 73.26%
